In [21]:
import os
from arcgis.features import GeoAccessor
from arcgis.geometry import Geometry

In [3]:
raw_gdb = r'D:\projects\geoai-retail\data\raw\raw.gdb'
blk_grp_fc = os.path.join(raw_gdb, 'block_groups')
blk_pt_fc = os.path.join(raw_gdb, 'block_points')

In [33]:
origin_df = GeoAccessor.from_featureclass(blk_grp_fc)
wgt_df = GeoAccessor.from_featureclass(blk_pt_fc)

In [37]:
def weighted_polygon_centroid(poly_df, wgt_df, poly_id_fld='ID'):
    
    # create a spatial index for both spatially enabled dataframes to speed up the join
    poly_df.spatial.sindex()
    wgt_df.spatial.sindex()

    # perform a spatial join between the points and the containing polygons
    join_df = wgt_df.spatial.join(poly_df)

    # extract the respective coordinates out of the geometry
    join_df['x'] = join_df['SHAPE'].apply(lambda geom: geom.centroid[0])
    join_df['y'] = join_df['SHAPE'].apply(lambda geom: geom.centroid[1])

    # get the id field following the join
    join_id_fld = poly_id_fld if poly_id_fld in join_df.columns else f'{poly_id_fld}_right'

    # calcuate the weighted centroid coordinates for each polygon and create geometries using these
    mean_df = join_df[[join_id_fld, 'x', 'y']].groupby(join_id_fld).mean()
    mean_df['SHAPE'] = mean_df.apply(lambda row: Geometry({'x': row['x'], 'y': row['y'], 'spatialReference': {'wkid': 4326}}), axis=1)

    # clean up the columns to a standard output schema
    mean_df.reset_index(inplace=True)
    mean_df = mean_df[[join_id_fld, 'SHAPE']].copy()
    mean_df.spatial.set_geometry('SHAPE')
    mean_df.columns = ['ID', 'SHAPE']
    
    return mean_df

In [40]:
%%time
mean_df = weighted_polygon_centroid(origin_df, wgt_df)
print(f'points - {len(wgt_df.index):,}')
print(f'polygons = {len(origin_df.index):,}')

points - 363,127
polygons = 6,963
Wall time: 2min 36s


In [41]:
mean_df.spatial.plot()

MapView(layout=Layout(height='400px', width='100%'))